In [1]:
from carbon.sdk import CarbonSDK, Tokens as T
from carbon import CarbonOrderUI, CarbonSimulatorUI
from carbon.helpers.widgets import CheckboxManager, DropdownManager, PcSliderManager
from matplotlib import pyplot as plt
import numpy as np
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonSDK))
!node --version

CarbonSDK v0.9 (11/Mar/2023)
v18.14.2


# Maker Functionality [NB20]


In order to start the server, please run

    source /Volumes/Private/bin/sdkserver
    node server/sdkserver.mjs

In [2]:
SDK = CarbonSDK(disclaimer=False, verbose=False, Tokens=T)
pairs, pairs_s = SDK.pairs(inclstr=True)
sdkaddr = SDK.addr()
print(sdkaddr)
SDK.version().get("msg")

0x02aA0c42702110747c4cEa3C4Daf4D6cF701a6Ee


'Carbon SDK v0.9 (7/Mar/2022)'

## Pair selection

In [3]:
try:
    pairs_sel = [s.strip() for s in pairs_s.split(",")]
    pairs_w()
except:
    pairs_w = DropdownManager(pairs_sel, defaultval="USDC/ETH")
    pairs_w()

Dropdown(options=(('USDC/ETH', 'USDC/ETH'),), value='USDC/ETH')

In [4]:
try:
    inverse_w()
except:
    inverse_w = CheckboxManager(["inverse"], values=[1])
    inverse_w()

In [5]:
tknb,tknq = pairs_w.value.split("/")
if inverse_w.values[0]:
    tknb,tknq = tknq, tknb
pair = f"{tknb}/{tknq}"
pair

'ETH/USDC'

## Market information

In [6]:
m = SDK.mGetMarketByPair(pair=pair)
m

MarketByPair(pair='ETH/USDC', bestBid=1599.9999998788553, bestAsk=2500.0, liqBid=2000.0, liqAsk=2.0, lastBid=1499.999999795006, lastAsk=2600.0000000000114)

In [7]:
SDK.roundsd(m.lastBid,6), SDK.roundsd(m.bestBid,6), SDK.roundsd(m.mid,6), SDK.roundsd(m.bestAsk,6), SDK.roundsd(m.lastAsk,6)

(1500.0, 1600.0, 2050.0, 2500.0, 2600.0)

In [8]:
(SDK.roundsd(m.liqBid,4), m.liqBidUnit, SDK.roundsd(m.liqAsk,4), m.liqAskUnit)

(2000.0, 'USDC', 2.0, 'ETH')

## Create strategy

#### `mCreateStrategy`

In [9]:
try:
    strat_w()
except:
    strat_w = PcSliderManager(choices=["buyStart", "buyEnd", "sellStart", "sellEnd"], values=[0, 1, 0, 1])
    strat_w()

In [10]:
sellAmountTknB =     1
buyAmountTknQ  =   100
x = strat_w.values
print(f"pair = {pair}, TknB={tknb}, TknQ={tknq}")
print(f"buy  {tknb} {SDK.roundsd(m.ibid(x[0]),6)} down to {SDK.roundsd(m.ibid(x[1]),6)}   [{sellAmountTknB} {tknq}] ")
print(f"sell {tknb} {SDK.roundsd(m.iask(x[2]),6)}      to {SDK.roundsd(m.iask(x[3]),6)}   [{buyAmountTknQ} {tknb}] ")

pair = ETH/USDC, TknB=ETH, TknQ=USDC
buy  ETH 1600.0 down to 1500.0   [1 USDC] 
sell ETH 2500.0      to 2600.0   [100 ETH] 


In [11]:
x = strat_w.values
tx = SDK.mCreateStrategy(
    pair, 
    buyRangeStart  = m.ibid(x[0]), buyRangeEnd  = m.ibid(x[1]), buyAmountTknQ = buyAmountTknQ, 
    sellRangeStart = m.iask(x[2]), sellRangeEnd = m.iask(x[3]), sellAmountTknB = sellAmountTknB, 
    overrides = None)
tx

{'data': '0xf727473a000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb480000000000000000000000000000000000000000000000000de0b6b3a76400000000000000000000000000000000000000000000000000000de0b6b3a76400000000000000000000000000000000000000000000000000000009c23178611340000000000000000000000000000000000000000000000000000f99373a1e7bb30000000000000000000000000000000000000000000000000000000005f5e1000000000000000000000000000000000000000000000000000000000005f5e10000000000000000000000000000000000000000000000000000000000154f52e10000000000000000000000000000000000000000000000000000000289c75e3a',
 'to': '0x135E72f1b65227b1C8e59DC1217096A114086876',
 'gasLimit': {'type': 'BigNumber', 'hex': '0x3b9ac9ff'},
 'value': {'type': 'BigNumber', 'hex': '0x0de0b6b3a7640000'}}

In [12]:
try:
    submit_w()
except:
    submit_w = CheckboxManager(["submit"], values=[0])
    submit_w()

In [13]:
if submit_w.values[0]: 
    r = SDK.signsubmittx(tx, sign=True)
else:
    r = "check box above to submit"
r

'check box above to submit'

## Review strategies

In [14]:
data = SDK.getUserStrategies(user=sdkaddr)
data

[{'id': {'type': 'BigNumber', 'hex': '0x0100000000000000000000000000000001'},
  'baseToken': '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE',
  'quoteToken': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
  'buyPriceLow': '1499.999999795006131230270249480266202369446401111707789510774091468192636966705322265625',
  'buyPriceHigh': '1599.99999987885530572613908948335161402097039096137365277172648347914218902587890625',
  'buyBudget': '2000',
  'sellPriceLow': '2500',
  'sellPriceHigh': '2600.000000000011469981186957619896170942620083990429606213754278889677960415165669457991666380074129',
  'sellBudget': '2',
  'encoded': {'id': {'type': 'BigNumber',
    'hex': '0x0100000000000000000000000000000001'},
   'token0': '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE',
   'token1': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
   'order0': {'y': {'type': 'BigNumber', 'hex': '0x1bc16d674ec80000'},
    'z': {'type': 'BigNumber', 'hex': '0x1bc16d674ec80000'},
    'A': {'type': 'BigNumber', 'hex': '0x09c

In [15]:
data = SDK.mGetUserStrategies(user=sdkaddr)
print("retrieved strategies", len(data))
s=data[0]
s

retrieved strategies 1


Strategy(sid=340282366920938463463374607431768211457, baseToken='0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', baseTokenName='ETH', quoteToken='0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', quoteTokenName='USDC', buyPriceLow=1500.0, buyPriceHigh=1600.0, buyBudget=2000.0, sellPriceLow=2500.0, sellPriceHigh=2600.0, sellBudget=2.0, encoded=EncodedStrategy(sid=340282366920938463463374607431768211457, order0=EncodedOrder(token='0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE', y=2000000000000000000, z=2000000000000000000, A=2746792519209792, B=4390587127987123), order1=EncodedOrder(token='0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48', y=2000000000, z=2000000000, A=357520097, B=10901478971)))

In [16]:
print(s.encoded.raw_json)
s.encoded.raw

{
    "id": "340282366920938463463374607431768211457",
    "token0": "0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE",
    "token1": "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48",
    "order0": {
        "y": "2000000000000000000",
        "z": "2000000000000000000",
        "A": "2746792519209792",
        "B": "4390587127987123"
    },
    "order1": {
        "y": "2000000000",
        "z": "2000000000",
        "A": "357520097",
        "B": "10901478971"
    }
}


{'id': '340282366920938463463374607431768211457',
 'token0': '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE',
 'token1': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
 'order0': {'y': '2000000000000000000',
  'z': '2000000000000000000',
  'A': '2746792519209792',
  'B': '4390587127987123'},
 'order1': {'y': '2000000000',
  'z': '2000000000',
  'A': '357520097',
  'B': '10901478971'}}

In [17]:
update = SDK.StrategyUpdate(buyPriceLow=1000, buyPriceHigh=1100, buyBudget=2000, sellPriceLow=3000, sellPriceHigh=3100, sellBudget=2)

In [18]:
r = SDK.updateStrategy(s.sid, s.encoded.raw_json, s.baseToken, s.quoteToken, update, sync=True)
r.json()

{'success': False,
 'error': "Cannot read properties of undefined (reading 'y')"}

In [22]:
SDK.StrategyUpdate(buyPriceLow=1000).asdict()

{'buyPriceLow': '1000'}

In [20]:
raise

RuntimeError: No active exception to reraise

In [ ]:
d, de=data[0], data[0].encoded
print(de.order1.descr)
print(de.descr[1])
print(d.descr[1])
f"buying {d.baseToken} @", de.order1.p_start, de.order1.p_end, f"{d.quoteToken} per {d.baseToken}"

In [ ]:
d, de=data[0], data[0].encoded
print(de.order0.descr)
print(de.descr[0])
print(d.descr[0])
f"selling {d.baseToken} @", 1/de.order0.p_start, 1/de.order0.p_end, f"{d.quoteToken} per {d.baseToken}"

In [ ]:
print(f"Number of strategies: {len(data)}")

In [ ]:
# Sim = CarbonSimulatorUI(verbose=False, raiseonerror=True)
# for d in data:
#     Sim.add_fromsdk(d, 6)
# Sim.state()["orders"]

## Delete strategies

In [ ]:
stratids = [d["id"] for d in data]
try:
    delete_w()
    assert len(data) == lendata
except:
    delete_w = CheckboxManager(stratids)
    lendata=len(data)
    delete_w()

In [ ]:
try:
    submitd_w()
except:
    submitd_w = CheckboxManager(["submit delete"], values=[0])
    submitd_w()

In [ ]:
for cid, ds in zip(stratids, delete_w.values):
    if ds:
        data = SDK.deleteStrategy(strategyId=cid)
        #print(data)
        if submitd_w.values[0]:
            result = SDK.signsubmittx(data, sign=True)
            print(f"Deleteting strategy {cid}: ", result)
            delete_w = None
        else:
            print(f"Would delete strategy {cid} if box was ticked ")

In [27]:
import json
dct = {
    1:"1",
    2:"2",
    "encoded": {1:1, 2:2}
}
dct

{1: '1', 2: '2', 'encoded': {1: 1, 2: 2}}